In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
from lwautils import lwa_arx 
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st


## Summary

I used this version of the notebook after updating the firmware to have an edge detect in the veto, to keep the effective window from being longer than desired.  I also added a register and a shared BRAM for monitoring the trigger rate right out of the coincidencer block, since I couldn't figure out why the received triggers register stayed zero.  In doing this set of tests with the new firmware, I realized that since the delays are set right after the ADC output, using the counter signal in the cosmic ray block always has all signals counting in sync no matter what delay I apply, so many of the tests below are not doing the intended test.  I am now compiling a new design with a counter test signal before the delay.  I should however be able to test the that triggers make it out of the block, by having no veto antennas, and I don't yet understand why this isn't working.

In [38]:
'''
def packantennaroles(roles_array):
    #turn an array of 64 ones and zeros into two 32 bit integers, one for the first half and one for the second
    output1=0
    output2=0
    for i in range(32):
        output1+=roles_array[i]<<i
        output2+=roles_array[i+32]<<i

    return output1, output2

def setup_coincidencer(casperbrd,trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      core_roles_array,
                      veto_roles_array):
    # set parameters
    setvalue(casperbrd,"trigger_power_thresh","cr_registers.xlsx",trigger_power_thresh)
    setvalue(casperbrd,"veto_power_thresh","cr_registers.xlsx",veto_power_thresh)
    setvalue(casperbrd,'trigger_window','cr_registers.xlsx',trigger_window) 
    setvalue(casperbrd,'veto_window','cr_registers.xlsx',veto_window) 
    setvalue(casperbrd,'antenna_number_thresh','cr_registers.xlsx',antenna_number_thresh) 
    setvalue(casperbrd,'veto_number_thresh','cr_registers.xlsx',veto_number_thresh) 
    #set the antenna roles
    core_roles1,core_roles2=packantennaroles(core_roles_array)
    veto_roles1,veto_roles2=packantennaroles(veto_roles_array)
    setvalue(casperbrd,'trigger_antennas1','cr_registers.xlsx',core_roles1)
    setvalue(casperbrd,'trigger_antennas2','cr_registers.xlsx',core_roles2)
    setvalue(casperbrd,'veto_antennas1','cr_registers.xlsx',veto_roles1)
    setvalue(casperbrd,'veto_antennas2','cr_registers.xlsx',veto_roles2)

    return

def set_delays(casperbrd,delays):
#see notebook subarray_threshold_scans for a function to extract the delays from a text file of delays and antenna names
    delayblock=blocks.delay.Delay(casperbrd, 'delay', n_streams=64, logger=None)
    delayblock.initialize()
    for i in range(64):
        z=delays[i]
        delayblock.set_delay(i,z)
'''
def test_coincidencer(casperbrd,trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      delays,
                      core_roles_array,
                      veto_roles_array):

    setup_coincidencer(casperbrd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)
    set_delays(casperbrd,delays)

    # reset trigger counter
    setvalue(casperbrd,"trig_debug_reset","cr_registers.xlsx",1)
    time.sleep(0.003)
    setvalue(casperbrd,"trig_debug_reset","cr_registers.xlsx",0)
    internal_trigger_count1=getvalue(casperbrd,"count_triggers","cr_registers.xlsx")

    time.sleep(2.75)  #have to wait for it to update
    trigger_count=getvalue(casperbrd,"count_triggers","cr_registers.xlsx")
    internal_trigger_count2=getvalue(casperbrd,"count_triggers","cr_registers.xlsx")
    triggers_leaving_block=(internal_trigger_count1-internal_trigger_count2)
    veto_coincidence,core_coincidence=struct.unpack('>2l',casperbrd.read("cosmic_ray_core_and_veto_rate",2*4,0))
    internal_triggers=struct.unpack('>2l',casperbrd.read("cosmic_ray_internal_trigger_rate",2*4,0))
    print(internal_triggers)
    print("Core coincidence per 10b cycle: ", core_coincidence/262144.)
    print("Veto coincidence per 10b cycle: ", veto_coincidence/262144.)
    print("Triggers per 10b cycle (roughly)",trigger_count/262144.)
    print("Triggers leaving coincidence_trigger block per cycle", internal_triggers)
    print("Triggers leaving block", triggers_leaving_block)
    return

0.55

In [3]:
fpgfile='/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-09-01_1917.fpg'
brdname='snap01'
brd=snap2_fengine.Snap2Fengine(brdname)
casperbrd = casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport)

2022-09-02 21:05:09,507 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-09-02 21:05:09,719 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1


In [4]:
brd.program(fpgfile)


2022-09-02 21:05:09,740 - lwa_f.snap2_fengine:snap01 - INFO - Loading firmware /home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-09-01_1917.fpg to snap01
2022-09-02 21:05:32,301 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-09-02 21:05:32,622 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1


In [5]:
casperbrd.listdev()
brd.initialize(read_only=False)
brd.sync.arm_sync()
brd.sync.sw_sync()

2022-09-02 21:05:32,647 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): fpga
2022-09-02 21:05:32,648 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): adc
2022-09-02 21:05:32,651 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: Setting clock source to 1
2022-09-02 21:05:32,659 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1: Setting clock source to 1
2022-09-02 21:05:32,665 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 0: Setting clock source to 1
2022-09-02 21:05:32,672 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 1: Setting clock source to 1
2022-09-02 21:05:41,278 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0 clock rates: [490926768, 98278734, 98369334, 0, 0]
2022-09-02 21:05:41,279 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: MMCM locked
2022-09-02 21:05:42,290 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1 clock rates: [490693885, 98191102, 98264715, 98281232, 0]
2022-09-

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-02 21:05:52,688 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[360, 352, 360, 344, 352, 344, 344, 344], [328, 312, 312, 312, 312, 304, 304, 312], [296, 296, 296, 296, 296, 288, 296, 304], [328, 344, 328, 328, 328, 328, 328, 328], [312, 312, 312, 320, 312, 312, 312, 304], [280, 288, 296, 288, 288, 280, 280, 280], [304, 280, 288, 288, 288, 304, 288, 288], [328, 328, 328, 328, 328, 336, 336, 320]]
2022-09-02 21:05:52,688 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 40, 40, 40, 32], [40, 40, 40, 40, 40, 32, 32, 40], [40, 40, 40, 40, 40, 32, 40, 32], [40, 40, 40, 40, 40, 32, 40, 32], [40, 40, 40, 40, 40, 40, 40, 32], [40, 40, 40, 40, 40, 40, 40, 40], [40, 32, 40, 32, 32, 32, 40, 40], [40, 40, 40, 40, 40, 40, 40, 32]]
2022-09-02 21:05:52,689 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXX
2022-09-02 21:05:52,690 - lwa_f.blocks.block:snap01 - INF

2022-09-02 21:05:52,723 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,724 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,724 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,725 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,725 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,726 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:05:52,727 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

100% (64 of 64) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


...

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-02 21:06:53,259 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 0 because nowhere was good
2022-09-02 21:06:53,271 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 1 because nowhere was good
2022-09-02 21:06:53,284 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 2 because nowhere was good
2022-09-02 21:06:53,295 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 3 because nowhere was good
2022-09-02 21:06:54,436 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-02 21:07:02,456 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [304, 312, 312, 320, 312, 312, 304, 312], [248, 248, 248, 240, 256, 240, 256, 240], [296, 288, 296, 304, 296, 288, 288, 296], [328, 328, 328, 328, 328, 328, 328, 320]]
2022-09-02 21:07:02,457 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [40, 40, 40, 40, 40, 40, 32, 40], [40, 40, 32, 32, 40, 32, 40, 32], [32, 32, 32, 32, 32, 32, 32, 32], [40, 32, 32, 40, 32, 40, 32, 32]]
2022-09-02 21:07:02,458 - lwa_f.blocks.block:snap01 - WARNING - adc - Delay solutions have small slack
2022-09-02 21:07:02,459 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,460 - lwa_f.blocks.block:snap0

2022-09-02 21:07:02,501 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 1:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,502 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,503 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,503 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,504 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,505 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXX
2022-09-02 21:07:02,506 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXX

In [6]:
setup_data_source(casperbrd,"counter")

Data source set to counter


The data source is set to a counter. 
## First test
The threshold is set to 511, all antennas have both roles, the antenna number thresholds and the window thresholds are zero.  As expected, all antennas reach the threshold once per 1024 clock cycles, and so there is a trigger and a veto once every 1024 clock cycles, or 262144 times in every time interval between updating the counter register.

In [39]:
#define parameters
trigger_power_thresh=511**2
veto_power_thresh=511**2
trigger_window=100
veto_window=200
antenna_number_thresh=0
veto_number_thresh=0

#see notebook subarray_threshold_scans for a function to extract the delays from a text file of delays and antenna names
delays=10*np.ones(64,dtype=int)
core_roles_array=np.ones(64,dtype=int)
veto_roles_array=np.ones(64,dtype=int)

setup_coincidencer(casperbrd,trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      core_roles_array,
                      veto_roles_array)
set_delays(casperbrd,delays)

time.sleep(2.75)  #have to wait for it to update
veto_coincidence,core_coincidence=struct.unpack('>2l',casperbrd.read("cosmic_ray_core_and_veto_rate",2*4,0))
core,veto = read_threshold_rates(casperbrd) #read each block

print("Core coincidence: ", core_coincidence)
print("Veto coincidence: ", veto_coincidence)
print(core[0],veto[1])
print(min(core),max(core))
print(min(veto),max(veto))
clockcycles_counted_for=(2**28)  #number of clock cycles that the trigger and veto rate counters counts for before updating
number_of_10bit_counter_cycles=clockcycles_counted_for/1024 #number of times the 10 bit counter starts over during the above count
print(clockcycles_counted_for)
print(number_of_10bit_counter_cycles)

Core coincidence:  262144
Veto coincidence:  262144
262144.0 262144.0
262144.0 262144.0
262144.0 262144.0
268435456
262144.0


In [40]:
#repeat above, in order to test out the test function
trigger_power_thresh=511**2
veto_power_thresh=511**2
trigger_window=100
veto_window=200
antenna_number_thresh=0
veto_number_thresh=0

set_delays(casperbrd,delays)
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)

(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0


# Main Tests

Trigger and veto power thresholds remain 511, and the core and veto number thresholds remain 2 (at least three antennas required).  The core antennas are signals 0, 1, and 2, and the veto antennas are signals 3, 4, 5.  The next sections go through five scenarios with variants.






In [41]:
trigger_power_thresh=511**2
veto_power_thresh=511**2
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:3]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[3:6]=1
antenna_number_thresh=2
veto_number_thresh=2

## 1. Veto after Core

### a) Veto long after

In [42]:
delays=np.ones(64,dtype=int)
delays[0:6]=[1,5,10,450,500,511] #core antennas are first three, veto antennas are second three
set_delays(casperbrd,delays)


print("1) Expect Trigger is vetoed")
trigger_windows=[11,50,100,200,300,400,500]
veto_windows=[511,450,400,300,200,100,62]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')


print("2) Expect Veto misses trigger")
trigger_windows=[11,20,100,11]
veto_windows=[62,100,300,350]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

print("3) Expect No trigger or veto")
trigger_windows=[10,9,2]
veto_windows=[61,60,49]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')


1) Expect Trigger is vetoed
trigger window:  11 veto window:  511
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.5
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  50 veto window:  450
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  100 veto window:  400
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  200 veto window:  300
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

tr

### Summary
There is never a trigger that makes it to the trigger counter, even when no veto is detected.  A trigger and a veto always ocurr, even when the window should be too short.  The first test puzzlingly has a veto half the time, maybe because the veto window nearly reaches the next cycle?

### Troubleshooting


(0, 0)


In [45]:
trigger_windows=[505,506,507,508,509,510,511]
veto_windows=[505,506,507,508,509,510,511]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

trigger window:  505 veto window:  505
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  506 veto window:  506
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  507 veto window:  507
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  508 veto window:  508
(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0

trigger window:  509 veto wi

In [12]:
trigger_windows=[340,341,342,343,345,347,350]
veto_windows=[340,341,342,343,345,347,350]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

trigger window:  340 veto window:  340
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  341 veto window:  341
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  342 veto window:  342
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  343 veto window:  343
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  345 veto window:  345
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  347 veto window:  347
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  350 veto window:  350
Core coincidence per 10b cycle:  1.0
Veto coincidence p

### b) Veto Soon after

### c) Veto widely arranged

## 2. Veto before core

### a) Veto long before


### b) Veto soon before

### c) Veto widely arranged

## 3. Veto antennas on either side of core

## 4. Veto antennas on either side and among core

## 5. Veto antennas among core

## Match the Simulink tests
Repeating all the tests in the August 23 Evernote note, but with the real system instead of Simulink.

A couple modifications to make an analogous test work with the 10 bit counter as signal:

--Voltage thresholds are always 511.
--Delays are 100,110,115,117,120 instead of 1000,1010,1015,1017,1020



In [13]:
delays=np.ones(64,dtype=int)
delays[0:5]=[100,110,115,117,120] #core antennas are first three, veto antennas are second three
trigger_power_thresh=511**2
veto_power_thresh=511**2

### Test A

In [14]:

core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=20
veto_window=7
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  There was a core and veto coincidence, matching the simulation, however there is no final trigger.

### Test B

In [15]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=7
veto_window=20
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test C

In [16]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=20
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test D

In [17]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=12
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test E

In [18]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[2:5]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[0:2]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=12
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, but the simulation did have a final trigger.

### Is there a trigger when I turn off the veto?

In [47]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:3]=1
veto_roles_array=np.zeros(64,dtype=int)
trigger_window=11
veto_window=100
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


(0, 0)
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0
Triggers leaving coincidence_trigger block per cycle (0, 0)
Triggers leaving block 0


## More NOtes
Antenna number thresholds are greater than. If 3 is set to the register, then four antennas are the minimum for a trigger.

In [20]:
(11<<16)+350

721246